In [13]:
import csv
import json
from datetime import datetime
import os

##### 
# DATA MODEL: Index Card

In [14]:
class Expense:
    def __init__(self, date, category, amount, description):    #Used to create expense object
        self.date = date
        self.category = category
        self.amount = amount
        self.description = description

    def ToDict(self):   #Used to write to csv file
        return {
            "date": self.date,
            "category": self.category,
            "amount": "{:.2f}".format(self.amount),
            "description": self.description,
        }

    def FromDict(data): #Used to read from csv file
        return Expense(
            date=data.get("date", ""),
            category=data.get("category", ""),
            amount=float(data.get("amount", 0.0)),
            description=data.get("description", ""),
        )


#####
# STORAGE LAYER: The Librarian

In [15]:
class ExpenseStorage:
    def __init__(self, csv_path="expenses.csv"): ## this defines the main csv file path as expenses.csv
        self.csv_path = csv_path

    def SaveExpensesToCsv(self, expenses): #Save expenses to a CSV file
        try: #error handling
            with open(self.csv_path, mode="w", newline="", encoding="utf-8") as f:  #open file - Write mode - newline handling - utf-8 encoding - 
                writer = csv.DictWriter(f, fieldnames=["date", "category", "amount", "description"])    #create dictionary matching the fieldnames which will be the column headers in csv file
                writer.writeheader() #writes header row
                for exp in expenses:
                    writer.writerow(exp.ToDict())   # writes each dictionary as a row in the csv file
            print("Saved {} expense(s) to '{}'.".format(len(expenses), self.csv_path))  #Testing print statement to confirm save
        except Exception as e:
            print("Error while saving to CSV: {}".format(e))

    def LoadExpensesFromCsv(self):
        expenses = [] #empty list to store expenses
        if not os.path.exists(self.csv_path): #check if file exists
            return expenses
        try:
            with open(self.csv_path, mode="r", newline="", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    try:
                        expense = Expense.FromDict(row)
                        expenses.append(expense)
                    except ValueError:
                        continue
        except Exception as e:
            print("Error while loading from CSV: {}".format(e))
        return expenses


##### 
# BUDGET MANAGER: Budgeting

In [16]:
class BudgetManager:
    def __init__(self, budget_json_path="budget.json"):
        self.budget_json_path = budget_json_path
        self.monthly_budget = 0.0

    def LoadBudgetOnStart(self):
        if not os.path.exists(self.budget_json_path):
            return
        try:
            with open(self.budget_json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                self.monthly_budget = float(data.get("monthly_budget", 0.0))
        except Exception as e:
            print("Error while loading budget: {}".format(e))

    def SaveBudget(self):
        try:
            with open(self.budget_json_path, "w", encoding="utf-8") as f:
                json.dump({"monthly_budget": self.monthly_budget}, f)
        except Exception as e:
            print("Error while saving budget: {}".format(e))

    def SetMonthlyBudget(self):
        while True:
            raw = input("Enter your monthly budget amount: ")
            try:
                value = float(raw)
                if value < 0:
                    print("Budget cannot be negative. Try again.")
                    continue
                self.monthly_budget = value
                self.SaveBudget()
                print("Monthly budget set to {:.2f}.".format(self.monthly_budget))
                break
            except ValueError:
                print("Please enter a valid number for the budget.")

    def TrackBudget(self, total_expenses):
        if self.monthly_budget <= 0:
            print("No monthly budget set yet. Please set one first.")
            return
        remaining = self.monthly_budget - total_expenses
        print("Total Expenses: {:.2f}".format(total_expenses))
        print("Monthly Budget: {:.2f}".format(self.monthly_budget))
        if remaining < 0:
            print("Warning: You have exceeded your budget!")
            print("Over Budget By: {:.2f}".format(abs(remaining)))
        else:
            print("You have {:.2f} left for the month.".format(remaining))


#####
# CORE TRACKER: Organizer

In [17]:
class ExpenseTracker:
    def __init__(self, storage, budget_manager):
        self.storage = storage
        self.budget_manager = budget_manager
        self.expenses = []

    def ValidateExpenseData(self, date_str, category, amount_str, description):
        if not date_str or not amount_str:
            return "All fields (date, category, amount, description) are required."
        try:
            datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            return "Date must be in YYYY-MM-DD format."
        try:
            amount_value = float(amount_str)
            if amount_value < 0:
                return "Amount cannot be negative."
        except ValueError:
            return "Amount must be a number."
        return None

    def AddExpenseInteractive(self):
        print("\nAdd a new expense:")
        date_str = input("Date (YYYY-MM-DD): ")
        category = input("Category: ")
        amount_str = input("Amount: ")
        description = input("Description: ")

        error = self.ValidateExpenseData(date_str, category, amount_str, description)
        if error:
            print("Error:", error)
            return

        amount_value = float(amount_str)
        expense = Expense(date_str, category, amount_value, description)
        self.expenses.append(expense)
        print("Expense added successfully.")

    def ViewExpenses(self):
        if not self.expenses:
            print("No expenses to show.")
            return
        print("\nYour Expenses:")
        for i, exp in enumerate(self.expenses, start=1):
            print(f"{i}. {exp.date} | {exp.category} | ${exp.amount:.2f} | {exp.description}")

    def DeleteExpense(self):
        if not self.expenses:
            print("No expenses to delete.")
            return
        self.ViewExpenses()
        try:
            index = int(input("Enter the expense number to delete: "))
            if 1 <= index <= len(self.expenses):
                removed = self.expenses.pop(index - 1)
                print(f"Deleted: {removed.date} | {removed.category} | ${removed.amount:.2f} | {removed.description}")
            else:
                print("Invalid number.")
        except ValueError:
            print("Please enter a valid integer.")

    def CalculateTotalExpenses(self):
        return sum(float(exp.amount) for exp in self.expenses)

    def CategorizeExpenseReport(self):
        if not self.expenses:
            print("No expenses to categorize.")
            return
        category_totals = {}
        for exp in self.expenses:
            cat = exp.category.strip() or "Uncategorized"
            category_totals[cat] = category_totals.get(cat, 0) + float(exp.amount)
        print("\nExpenses by Category:")
        for cat, total in sorted(category_totals.items()):
            print(f"- {cat}: ${total:.2f}")

    def SaveExpenses(self):
        self.storage.SaveExpensesToCsv(self.expenses)

    def LoadExpenses(self):
        self.expenses = self.storage.LoadExpensesFromCsv()


#####
# Main()

In [18]:
class ExpenseApp:
    def __init__(self):
        self.storage = ExpenseStorage()
        self.budget_manager = BudgetManager()
        self.tracker = ExpenseTracker(self.storage, self.budget_manager)

    def Start(self):
        self.tracker.LoadExpenses()
        self.budget_manager.LoadBudgetOnStart()
        print("Welcome to Personal Expense Tracker!\n")
        self.RunInteractiveMenu()

    def ShowMenu(self):
        print("\nMenu:")
        print("1) Add Expense")
        print("2) View Expenses")
        print("3) Delete Expense")
        print("4) Set Monthly Budget")
        print("5) Track Budget")
        print("6) Category Report")
        print("7) Save Expenses")
        print("8) Exit")

    def RunInteractiveMenu(self):
        while True:
            self.ShowMenu()
            choice = input("Choose (1–8): ")
            if choice == "1":
                self.tracker.AddExpenseInteractive()
            elif choice == "2":
                self.tracker.ViewExpenses()
            elif choice == "3":
                self.tracker.DeleteExpense()
            elif choice == "4":
                self.budget_manager.SetMonthlyBudget()
            elif choice == "5":
                total = self.tracker.CalculateTotalExpenses()
                self.budget_manager.TrackBudget(total)
            elif choice == "6":
                self.tracker.CategorizeExpenseReport()
            elif choice == "7":
                self.tracker.SaveExpenses()
            elif choice == "8":
                self.tracker.SaveExpenses()
                print("Goodbye 👋")
                break
            else:
                print("Invalid choice.")


#####
# ENTRY POINT: Code that runs when file is executed directly

In [19]:
app = ExpenseApp()
app.Start()  # launches the menu interactively

Welcome to Personal Expense Tracker!


Menu:
1) Add Expense
2) View Expenses
3) Delete Expense
4) Set Monthly Budget
5) Track Budget
6) Category Report
7) Save Expenses
8) Exit
Saved 101 expense(s) to 'expenses.csv'.
Goodbye 👋
